In [1]:
import os

# get a list of all files in the directory
directory = "Humor,Hist,Media,Food/"
files = os.listdir(directory)

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def preprocess(text):
    tokenizer = nltk.RegexpTokenizer(r"\w+")    # remove punctuations
    tokens = tokenizer.tokenize(text)    # token the text

    tokens = [token.lower() for token in tokens]    # lower case
    tokens_no_sw = [word for word in tokens if not word in stop_words]    # remove stop words
    return tokens_no_sw
    # return tokens

In [3]:
token_doc_id = {}
token_freq = {}

for file_name in files:    # for each file
    file_text = open(directory+file_name,'r',encoding='ISO-8859-1').read()    # read the file
    tokens = preprocess(file_text)
    # print(tokens)
    for token in tokens:
        if token not in token_doc_id:
            token_doc_id[token] = {}
            token_freq[token] = 0
        if file_name not in token_doc_id[token]:
            token_doc_id[token][file_name] = 0    # add file name to token list
        token_doc_id[token][file_name] += 1    # update word frequency in file
        token_freq[token]+=1

In [4]:
def and_op(list_left,list_right,comparisons):
    # get intersection of left and right
    intersection = []
    i = 0
    j = 0
    while i<len(list_left) and j<len(list_right):
        if list_left[i]==list_right[j]:    # add when present in both lists
            intersection.append(list_left[i])
            i+=1
            j+=1
        elif list_left[i]<list_right[j]:
            i+=1
        else:
            j+=1
        comparisons+=1
    return intersection,comparisons

def or_op(list_left,list_right,comparisons):
    # get union of left and right
    union = []
    i = 0
    j = 0
    while i<len(list_left) and j<len(list_right):
        if list_left[i]==list_right[j]:    # adding common element only once
            union.append(list_left[i])
            i+=1
            j+=1
        elif list_left[i]<list_right[j]:
            union.append(list_left[i])
            i+=1
        else:
            union.append(list_right[j])
            j+=1
        comparisons+=1

    while i<len(list_left):
        union.append(list_left[i])
        i+=1
    while j<len(list_right):
        union.append(list_right[j])
        j+=1
        
    return union,comparisons

def not_op(word_docs):
    # get complement of given list
    complement = []
    for f in files:
        if f not in word_docs:
            complement.append(f)
    return sorted(complement)

In [5]:
def do_operation(query,opers):
    # do query and operations
    num_ops = len(opers)
    query = query[:num_ops+1]
    # print(query,opers)

    # combining query words and operations into one sequence
    query_doc_id = [sorted(list(token_doc_id[query[0]].keys()))]
    input_query_final = [query[0]]
    i = 0
    while i<len(opers):
        input_query_final.append(opers[i])
        input_query_final.append(query[i+1])
        query_doc_id.append(opers[i])
        query_doc_id.append(sorted(list(token_doc_id[query[i+1]].keys())))
        i+=1
    print("Query input ="," ".join(input_query_final))
    # for i in query_doc_id: print(i)

    # taking NOT of lists
    for i in range(len(query_doc_id)):
        if query_doc_id[i]=='AND NOT' or query_doc_id[i]=='OR NOT':
            query_doc_id[i+1] = not_op(query_doc_id[i+1])
    # for i in query_doc_id: print(i)

    # doing AND operation before, because of higher precedence
    # and
    i = 0
    comparisons = 0
    while i<len(query_doc_id)-1:
        if query_doc_id[i]=='AND' or query_doc_id[i]=='AND NOT':
            # if there is AND at ith position, do AND operation on (i-1)th and (i+1)th lists
            # store its result at (i-1)th position and remove ith and (i+1)th elements
            query_doc_id[i-1],comparisons = and_op(query_doc_id[i-1],query_doc_id[i+1],comparisons)
            query_doc_id.pop(i)
            query_doc_id.pop(i)
        else: i+=1
    # for i in query_doc_id: print(i)

    # OR operation
    # or
    i = 0
    while i<len(query_doc_id)-1:
        if query_doc_id[i]=='OR' or query_doc_id[i]=='OR NOT':
            # if there is OR at ith position, do OR operation on (i-1)th and (i+1)th lists
            # store its result at (i-1)th position and remove ith and (i+1)th elements
            query_doc_id[i-1],comparisons = or_op(query_doc_id[i-1],query_doc_id[i+1],comparisons)
            query_doc_id.pop(i)
            query_doc_id.pop(i)
        else: i+=1
    # for i in query_doc_id: print(i)
    print('number of documents retrieved -',len(set(query_doc_id[0])))
    print('number of comparisons =',comparisons)
    return sorted(list(set(query_doc_id[0])))

In [14]:
def preprocess_operations(ops):
    return [op.upper() for op in ops]

# query
with open('Q1_input.txt','r') as inp:
    number_of_queries = int(inp.readline())
    for i in range(number_of_queries):
        input_text = inp.readline()
        input_ops = inp.readline()[:-1].split(", ")
        ops_pp = preprocess_operations(input_ops)
        input_text = preprocess(input_text)
        # print(input_text,ops_pp)
        print(do_operation(input_text,ops_pp))
        print()


Query input = lion OR stood OR thoughtfully OR moment
number of documents retrieved - 185
number of comparisons = 366
['a_tv_t-p.com', 'ambrose.bie', 'anim_lif.txt', 'anime.lif', 'annoy.fascist', 'art-fart.hum', 'b-2.jok', 'barney.txt', 'bbh_intv.txt', 'beauty.tm', 'beesherb.txt', 'bitnet.txt', 'bmdn01.txt', 'boneles2.txt', 'booze1.fun', 'butwrong.hum', 'bw-phwan.hat', 'bw.txt', 'cabbage.txt', 'caesardr.sal', 'calculus.txt', 'candy.txt', 'chickenheadbbs.txt', 'childhoo.jok', 'christop.int', 'clancy.txt', 'cmu.share', 'cogdis.txt', 'collected_quotes.txt', 'computer.txt', 'conan.txt', 'consp.txt', 'cookie.1', 'coyote.txt', 'cuchy.hum', 'cybrtrsh.txt', 'dead4.txt', 'dead5.txt', 'deep.txt', 'devils.jok', 'doggun.sto', 'drinks.gui', 'dromes.txt', 'dthought.txt', 'econridl.fun', 'engineer.hum', 'english.txt', 'epi_.txt', 'epi_tton.txt', 'epitaph', 'eskimo.nel', 'exam.50', 'facedeth.txt', 'fascist.txt', 'filmgoof.txt', 'flux_fix.txt', 'fuckyou2.txt', 'gas.txt', 'gd_gal.txt', 'gd_ql.txt', 'gho

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c98dd744-d9d5-4215-badf-4f45241c19af' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>